In [129]:
import pandas as pd

In [130]:
# load all the datasets
merchantData = pd.read_csv("dataset/train_merchant_data.csv")#train_merchant_data.csv")
ipRangeData = pd.read_csv("dataset/ip_boundaries_countries.csv")
ordersData = pd.read_csv("dataset/train_order_data.csv")#train_order_data.csv")

### Describe merchant data

In [131]:
# list all the info
print('-- Dataset information --')
print(merchantData.info())

# show the data description for numeric columns
merchantData.describe()

-- Dataset information --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54213 entries, 0 to 54212
Data columns (total 7 columns):
Ecommerce_Provider_ID         54213 non-null int64
Merchant_ID                   54213 non-null int64
Merchant_Registration_Date    54213 non-null object
Registered_Device_ID          54213 non-null object
Gender                        54213 non-null object
Age                           54213 non-null int64
IP_Address                    54213 non-null object
dtypes: int64(3), object(4)
memory usage: 2.9+ MB
None


,Ecommerce_Provider_ID,Merchant_ID,Age
count,54213.0,54213.000000,54213.000000
mean,1746213.0,200395.176212,33.122240
std,0.0,115398.486895,8.630091
min,1746213.0,2.000000,18.000000
25%,1746213.0,100997.000000,27.000000
50%,1746213.0,200574.000000,32.000000
75%,1746213.0,300407.000000,39.000000
max,1746213.0,399993.000000,72.000000


### Describe Orders data

In [132]:
# list all the info
print('-- Dataset information --')
print(ordersData.info())

# show the data description for numeric columns
ordersData.describe()

-- Dataset information --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54213 entries, 0 to 54212
Data columns (total 7 columns):
Customer_ID             54213 non-null int64
Order_ID                54213 non-null object
Date_of_Order           54213 non-null object
Order_Value_USD         54213 non-null int64
Order_Source            54213 non-null object
Order_Payment_Method    54213 non-null object
Merchant_ID             54213 non-null int64
dtypes: int64(3), object(4)
memory usage: 2.9+ MB
None


,Customer_ID,Order_Value_USD,Merchant_ID
count,54213.000000,54213.000000,54213.000000
mean,137966.285208,92.230240,200395.176212
std,15563.516156,45.673263,115398.486895
min,111234.000000,22.000000,2.000000
25%,124471.000000,55.000000,100997.000000
50%,137864.000000,88.000000,200574.000000
75%,151405.000000,122.000000,300407.000000
max,165000.000000,385.000000,399993.000000


### Describe IP dataset

In [133]:
# list all the info
print('-- Dataset information --')
print(ipRangeData.info())

# show the data description for numeric columns
ipRangeData.head()

-- Dataset information --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
lower_bound_ip_address    138846 non-null object
upper_bound_ip_address    138846 non-null object
country                   138846 non-null object
dtypes: object(3)
memory usage: 3.2+ MB
None


,lower_bound_ip_address,upper_bound_ip_address,country
0,1.0.0.0,1.0.0.255,Australia
1,1.0.1.0,1.0.1.255,China
2,1.0.2.0,1.0.3.255,China
3,1.0.4.0,1.0.7.255,Australia
4,1.0.8.0,1.0.15.255,China


### Merge Merchant and IP range data to extract Country

In [134]:
# step 1 : convert all IP address to numeric value for easier comprasion
from netaddr import *

for i_index, ipRow in ipRangeData.iterrows():
    ipRow['lower_bound_ip_address'] = int(IPAddress(ipRow['lower_bound_ip_address']))
    ipRow['upper_bound_ip_address'] = int(IPAddress(ipRow['upper_bound_ip_address']))

In [135]:
# step 2 : convert to proper datatype for the new column values...
ipRangeData['lower_bound_ip_address'] = ipRangeData['lower_bound_ip_address'].astype('int64')
ipRangeData['upper_bound_ip_address'] = ipRangeData['upper_bound_ip_address'].astype('int64')

In [136]:
ipRangeData.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216,16777471,Australia
1,16777472,16777727,China
2,16777728,16778239,China
3,16778240,16779263,Australia
4,16779264,16781311,China


In [137]:
# step 3 : define a function to get country
def getCountry(row):
    ip = int(IPAddress(row['IP_Address']))
    countryDetails = ipRangeData.loc[(ip >= ipRangeData['lower_bound_ip_address']) & 
                             (ip <= ipRangeData['upper_bound_ip_address']),:]
    if(len(countryDetails) > 0):
        row['country'] = countryDetails.iloc[0,2]
    else:
        row['country'] = 'InvalidIP'
    return row

In [138]:
# Apply a user defined function to get country
merchantDataWithCountry = merchantData.apply(getCountry, axis=1)

In [139]:
merchantDataWithCountry.head()

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,country
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,United States
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,United Kingdom
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,Japan
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,Austria
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,China


### Now merge merchant with orders data

In [140]:
finalDataset = pd.merge(merchantDataWithCountry, ordersData, on='Merchant_ID')
finalDataset.head()

,Ecommerce_Provider_ID,Merchant_ID,Merchant_Registration_Date,Registered_Device_ID,Gender,Age,IP_Address,country,Customer_ID,Order_ID,Date_of_Order,Order_Value_USD,Order_Source,Order_Payment_Method
0,1746213,50448,2018-05-01 21:15:11,VATQMMZTVOZUT,F,39,48.151.136.76,United States,129697,3b8983a83c7b,2018-07-30 10:59:13,90,SEO,Debit Card
1,1746213,338754,2018-04-14 10:13:00,LJCILLBRQZNKS,M,35,94.9.145.169,United Kingdom,117390,34b5eb921228,2018-06-15 11:19:47,98,SEO,Internet Banking
2,1746213,291127,2018-06-20 07:44:22,JFVHSUGKDAYZV,F,40,58.94.157.121,Japan,120162,41a1c86ff08b,2018-08-13 10:06:26,95,SEO,Credit Card
3,1746213,319919,2018-06-27 01:41:39,WFRXMPLQYXRMY,M,37,193.187.41.186,Austria,128228,e8c3ad80d916,2018-07-22 15:46:51,100,Direct,E-wallet
4,1746213,195911,2018-01-05 00:55:41,GGHKWMSWHCMID,F,27,125.96.20.172,China,136029,e71ab1f26785,2018-04-16 08:02:44,78,SEO,E-wallet


### Write to file the merged data

In [141]:
finalDataset.to_csv('finalTestDataset.csv',sep=',', encoding='utf-8', index=False)